### 分析SelfKG中failure case的分布


In [ ]:
# coding: UTF-8
import argparse
import logging
import os
import random
from datetime import datetime
from posixpath import join

import faiss
import numpy as np
import pandas as pd
# using labse
# from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torch.nn import *

from loader.DBP15KRawNeighbors import DBP15KRawNeighbors
from script.preprocess.deal_raw_dataset import MyRawdataset
from settings import *

In [ ]:
#读取ent_ids转换成id：url的字典
ent_id = "../../Datasets/dbp15k/dbp15k-fr-en-tgt/ent_ids_2"

id2enturl = {}
with open(ent_id, 'r', encoding='utf-8') as f:
    for line in f:
        id_entity = line.strip().split('\t')
        id2enturl[int(id_entity[0])] = id_entity[1].strip()

In [ ]:
#得到head rel tail的url
from os.path import join
from tqdm import *


triple_path = "../../Datasets/dbp15k/dbp15k-fr-en-tgt/triples_2"
path = "../../Datasets/dbp15k/dbp15k-fr-en-tgt/"
name = "rel_triples_whole_2"
h_2_t2rel_url = {}
with open(join(path, name), 'r', encoding='utf-8') as f:
    for line in tqdm(f):
        h_r_t_url = line.strip().split('\t')
        h_url = h_r_t_url[0]
        try:
            r_url = h_r_t_url[1]
        except:
            continue
        t_url = h_r_t_url[2]
        if h_url in id2enturl.values() and t_url in id2enturl.values():
            if not h_url in h_2_t2rel_url.keys():
                h_2_t2rel_url[h_url] = {}
#这个组织形式是head实体包含的所有triple，跟各个tail的关系。是这个head的tuple，是一跳的邻居
            h_2_t2rel_url[h_url][t_url] = r_url

In [ ]:
def fix_seed(seed=37):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.